In [ ]:
import qiskit
import numpy as np
from qiskit.quantum_info import Statevector, partial_trace
from music21 import stream, note, chord, instrument

# --- Global settings ---
GATE_DURATION = 2.0  # Set all musical note/rest durations
PENTATONIC = [0, 2, 4, 7, 9]  # C major pentatonic: C, D, E, G, A

# --- Helper functions ---

def quantize_to_pentatonic(midi_note):
    base = 60  # Middle C
    rel = midi_note - base
    octave = rel // 12
    semitone = rel % 12
    closest = min(PENTATONIC, key=lambda x: abs(x - semitone))
    return base + octave * 12 + closest

def state_to_bloch_angles(alpha, beta):
    theta = 2 * np.arccos(np.abs(alpha))
    phi = np.angle(beta) - np.angle(alpha)
    return theta, phi

def theta_to_note(theta):
    pitch = int((theta / np.pi) * 12) + 60
    return quantize_to_pentatonic(pitch)

def phi_to_note(phi):
    pitch = int(((phi + np.pi) / (2 * np.pi)) * 12) + 72
    return quantize_to_pentatonic(pitch)

def qubit_to_instrument(index):
    instruments = [instrument.Flute(), instrument.Piano(), instrument.Violin(),
                   instrument.Guitar(), instrument.Clarinet(), instrument.Trumpet()]
    return instruments[index % len(instruments)]

def apply_gate(state, gate, qargs, circuit):
    temp_circuit = qiskit.QuantumCircuit(circuit.num_qubits)
    qubit_indices = [circuit.qubits.index(qarg) for qarg in qargs]
    temp_circuit.append(gate, qubit_indices)
    return state.evolve(temp_circuit)

# --- Main: Convert quantum circuit to music ---
def circuit_to_music_explicit(circuit):
    num_qubits = circuit.num_qubits
    state = Statevector.from_label('0' * num_qubits)
    qubit_states = {i: [] for i in range(num_qubits)}
    music_score = stream.Score()

    for inst, qargs, _ in circuit.data:
        if inst.name == 'barrier':
            for i in range(num_qubits):
                reduced = partial_trace(state, [j for j in range(num_qubits) if j != i])
                alpha, beta = reduced.data[0][0], reduced.data[1][0]
                qubit_states[i].append((alpha, beta))
        elif inst.name == 'id':
            for i in range(num_qubits):
                qubit_states[i].append('rest')
        else:
            state = apply_gate(state, inst, qargs, circuit)

    for i in range(num_qubits):
        part = stream.Part()
        part.insert(0, qubit_to_instrument(i))
        for item in qubit_states[i]:
            if item == 'rest':
                r = note.Rest(quarterLength=GATE_DURATION)
                part.append(r)
            else:
                alpha, beta = item
                theta, phi = state_to_bloch_angles(alpha, beta)
                ch = chord.Chord([theta_to_note(theta), phi_to_note(phi)])
                ch.quarterLength = GATE_DURATION
                part.append(ch)
        music_score.append(part)

    return music_score, qubit_states

# --- Choose your quantum circuit ---

# 1️⃣ Superposition (1 qubit)
qc = qiskit.QuantumCircuit(1)
qc.h(0)
qc.barrier()
qc.rx(np.pi/4, 0)
qc.barrier()
qc.ry(np.pi/3, 0)
qc.barrier()
qc.rz(np.pi/2, 0)
qc.barrier()
qc.id(0)  # Add a rest
qc.barrier()

# 2️⃣ Entanglement (2 qubits — Bell pair)
# Uncomment below to test
# qc = qiskit.QuantumCircuit(2)
# qc.h(0)
# qc.cx(0, 1)
# qc.barrier()
# qc.rx(np.pi/4, 0)
# qc.barrier()
# qc.ry(np.pi/3, 0)
# qc.barrier()
# qc.rz(np.pi/2, 0)
# qc.barrier()
# qc.id(0)
# qc.id(1)
# qc.barrier()

# --- Generate music and save ---
music_score, qubit_states = circuit_to_music_explicit(qc)

music_score.show('text')  # Display symbolic notes
music_score.write('midi', fp='quantum_music_penta_simple.mid')  # Export to MIDI file


{0.0} <music21.stream.Part 0x123555940>
    {0.0} <music21.instrument.Flute 'Flute'>
    {0.0} <music21.chord.Chord G4 G5>
    {2.0} <music21.chord.Chord G4 G5>
    {4.0} <music21.chord.Chord A4 G5>
    {6.0} <music21.chord.Chord A4 A5>
    {8.0} <music21.note.Rest half>
    {10.0} <music21.chord.Chord A4 A5>


'quantum_music_penta_simple.mid'